# COVID-19 data in Poland

plotting data for Poland based on https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

## Preparation of data
to be run once a day

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
#import timeit
#pd.show_versions(as_json=False)

def prepare_data(name, url, i=73):
    df = pd.read_csv(url, parse_dates=True)
    df.rename(columns={'Country/Region': 'Region'}, inplace=True)    
    p = df[df.Region == "Poland"].drop(['Region', 'Province/State', 'Lat', 'Long'], axis=1)
    col = p.rename(index=lambda x: name).transpose()
    col.rename(index=lambda i: datetime.datetime.strptime(i, '%m/%d/%y'), inplace = True)
    return col[col.index >= datetime.datetime(2020, 3, 3)]

def calculate_mean(df, mean_window_size):
    return df.rolling(window=mean_window_size).mean().rename(columns=lambda x:"mean(%i) %s" % (mean_window_size,x))

#https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
c = prepare_data("confirmed", "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
d = prepare_data("deaths", "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
r = prepare_data("recovered", "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

p = c.merge(d, left_index = True, right_index = True).merge(r, left_index = True, right_index = True)
np = p.rolling(window=2).apply(lambda x: x[1] - x[0]).rename(columns=lambda x: "new "+x)

## Playing with data

In [2]:
npm2 = calculate_mean(np, 2)
npm3 = calculate_mean(np, 3)
npm4 = calculate_mean(np, 4)
npm7 = calculate_mean(np, 7)

## Presentation of data

In [3]:
p.merge(np.merge(npm2, left_index = True, right_index = True), left_index = True, right_index = True).tail(5)

In [5]:
fig, axes = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(15,15))
p.plot(ax=axes[0,0])
np.plot(ax=axes[0,1])
npm2.plot(ax=axes[1,0])
npm3.plot(ax=axes[1,1])
npm4.plot(ax=axes[2,0])
npm7.plot(ax=axes[2,1])